In [6]:
from nnsight import LanguageModel
import torch
import pandas as pd
from tqdm import tqdm
import numpy as np
import pickle
import sys

In [8]:
og = pd.read_csv('/home/gridsan/arunas/broca/ngs.csv')
print(og.columns)

Index(['Unnamed: 0', 'sentence', 'subordinate-sentence', 'passive-sentence',
       'it', 'it-r-1-null_subject', 'it-r-2-passive', 'it-r-3-subordinate',
       'it-u-1-negation', 'it-u-2-invert', 'it-u-3-gender', 'jp-r-1-sov',
       'jp-r-2-passive', 'jp-r-3-subordinate', 'jp-u-1-negation',
       'jp-u-2-invert', 'jp-u-3-past-tense', 'ng-sentence',
       'ng-subordinate-sentence', 'ng-passive-sentence', 'ng-it',
       'ng-it-r-1-null_subject', 'ng-it-r-2-passive', 'ng-it-r-3-subordinate',
       'ng-it-u-1-negation', 'ng-it-u-2-invert', 'ng-it-u-3-gender',
       'ng-jp-r-1-sov', 'ng-jp-r-2-passive', 'ng-jp-r-3-subordinate',
       'ng-jp-u-1-negation', 'ng-jp-u-2-invert', 'ng-jp-u-3-past-tense'],
      dtype='object')


In [9]:
device='cuda:0'
model = LanguageModel("/home/gridsan/arunas/models/mistralai/Mistral-7B-v0.1/",  load_in_4bit=True, dispatch=True, device_map=device) # Load the model

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

In [12]:
def get_prompt_from_df(filename):
    data = list(pd.read_csv(filename)['prompt'])
    data = [sentence.strip() for sentence in data]
    data = [sentence for sentence in data if not sentence == '']
    data = [sentence.replace('</s>', '\n') for sentence in data]
    golds = [sentence.strip().split("\n")[-1].strip().split('A:')[-1].strip() for sentence in data]
    data = [sentence[: -len(golds[idx])].strip() for idx, sentence in enumerate(data)]
    return data, golds

types = ['it', 'jp-r-2-passive', 'it-r-1-null_subject', 'jp-r-3-subordinate', 'it-r-2-passive', 'jp-u-1-negation', 'it-r-3-subordinate', 'jp-u-2-invert', 'it-u-1-negation', 'jp-u-3-past-tense', 'it-u-2-invert', 'passive-sentence', 'it-u-3-gender', 'sentence', 'jp-r-1-sov', 'subordinate-sentence']
sType=types[int(sys.argv[1])]

mlp_effects_cache = torch.zeros((model.config.num_hidden_layers, model.config.hidden_size)).to("cuda")
attn_effects_cache = torch.zeros((model.config.num_hidden_layers, model.config.hidden_size)).to("cuda")

In [40]:
sys.argv = []
def attrPatching(fullPrompt, gold):
    attn_layer_cache_prompt = {}
    mlp_layer_cache_prompt = {}

    attn_layer_cache_patch = {}
    mlp_layer_cache_patch = {}
    assert (gold == 'Yes' or gold == 'No')
    if (gold == 'Yes'):
        predictionExample = fullPrompt[fullPrompt[:-2].rfind(':')+1:-2].strip()
        patch = og[og[sType] == predictionExample][f"ng-{sType}"].iloc[0]
        patchPrompt = fullPrompt.replace(predictionExample, patch)
    else:
        patchPrompt = fullPrompt
        patch = fullPrompt[fullPrompt[:-2].rfind(':')+1:-2].strip()
        predictionExample = og[og[f"ng-{sType}"] == patch][sType].iloc[0]
        fullPrompt = patchPrompt.replace(patch, predictionExample)
        gold = "Yes"

    notGold = "No"
    gold = model.tokenizer(gold)["input_ids"]
    notGold = model.tokenizer(notGold)["input_ids"]
    with model.forward(inference=False) as runner:
        with runner.invoke(fullPrompt) as invoker:
            for layer in range(len(model.model.layers)):
                self_attn = model.model.layers[layer].self_attn.o_proj.output
                mlp = model.model.layers[layer].mlp.down_proj.output

                attn_layer_cache_prompt[layer] = {"forward": self_attn.detach().save(), "backward": self_attn.grad.detach().save()}
                mlp_layer_cache_prompt[layer] = {"forward": mlp.detach().save(), "backward": mlp.grad.detach().save()}

            logits = model.lm_head.output[:, -1, notGold] - model.lm_head.output[:, -1, gold]
            loss = logits.sum()
            loss.backward(retain_graph=False)

    with model.forward(inference=False) as runner:
        with runner.invoke(patchPrompt) as invoker:
            for layer in range(len(model.model.layers)):
                self_attn = model.model.layers[layer].self_attn.o_proj.output
                mlp = model.model.layers[layer].mlp.down_proj.output

                attn_layer_cache_patch[layer] = {"forward": self_attn.detach().save()}
                mlp_layer_cache_patch[layer] = {"forward": mlp.detach().save()}

    for layer in range(len(model.model.layers)):
        mlp_effects = mlp_layer_cache_prompt[layer]["backward"].value * (mlp_layer_cache_patch[layer]["forward"].value - mlp_layer_cache_prompt[layer]["forward"].value)
        attn_effects = attn_layer_cache_prompt[layer]["backward"].value * (attn_layer_cache_patch[layer]["forward"].value - attn_layer_cache_prompt[layer]["forward"].value)

        mlp_effects = mlp_effects[:, -1, :] # batch, token, hidden_states
        attn_effects = attn_effects[:, -1, :] # batch, token, hidden_states

        mlp_effects_cache[layer] += mlp_effects[0]
        attn_effects_cache[layer] += attn_effects[0]

prompts, golds = get_prompt_from_df(f'/home/gridsan/arunas/broca/llama/llama-prompt-outputs/llama-classification-train-test-det-{sType}-new.csv')


In [41]:
count  = 0
for prompt,gold in tqdm(zip(prompts, golds)):
    count += 1
    attrPatching(prompt, gold)

0it [00:00, ?it/s]

Q: Is this sentence grammatical? Yes or No: kon kon reads boy chapter
A: No

Q: Is this sentence grammatical? Yes or No: kon cat holds kon mouse
A: Yes

Q: Is this sentence grammatical? Yes or No: kons teachers hold kar bottle
A: Yes

Q: Is this sentence grammatical? Yes or No: kons teachers carry kar bottle
A: Yes

Q: Is this sentence grammatical? Yes or No: kon man eats kar pizza
A: Yes

Q: Is this sentence grammatical? Yes or No: kon carries cat kon mouse
A: No

Q: Is this sentence grammatical? Yes or No: carry teachers kons kon cap
A: No

Q: Is this sentence grammatical? Yes or No: kons cats bring kar bottle
A: Yes

Q: Is this sentence grammatical? Yes or No: cats kons climb kar tree
A: No

Q: Is this sentence grammatical? Yes or No: kons boys take kar box
A:


1it [00:17, 17.59s/it]

Q: Is this sentence grammatical? Yes or No: kons boys take bulb kar
A: No

Q: Is this sentence grammatical? Yes or No: kon cat holds toy kon
A: No

Q: Is this sentence grammatical? Yes or No: cats kons hold kon fish
A: No

Q: Is this sentence grammatical? Yes or No: kons men eat kon fish
A: Yes

Q: Is this sentence grammatical? Yes or No: kons cats take kon mouse
A: Yes

Q: Is this sentence grammatical? Yes or No: kons teachers toy kon carry
A: No

Q: Is this sentence grammatical? Yes or No: kon poet reads kon chapter
A: Yes

Q: Is this sentence grammatical? Yes or No: man kon carries kar box
A: No

Q: Is this sentence grammatical? Yes or No: kars climb girls kar mountain
A: No

Q: Is this sentence grammatical? Yes or No: roof dogs climb kon kons
A:


2it [00:35, 17.64s/it]

Q: Is this sentence grammatical? Yes or No: boys kons hold kar box
A: No

Q: Is this sentence grammatical? Yes or No: kons men bring kon fish
A: Yes

Q: Is this sentence grammatical? Yes or No: woman kar climbs kon roof
A: No

Q: Is this sentence grammatical? Yes or No: brings girl kar kar bulb
A: No

Q: Is this sentence grammatical? Yes or No: kons boys carry kar box
A: Yes

Q: Is this sentence grammatical? Yes or No: carry dogs kons kon mouse
A: No

Q: Is this sentence grammatical? Yes or No: kars kar eat girls pear
A: No

Q: Is this sentence grammatical? Yes or No: kar girl holds kar bulb
A: Yes

Q: Is this sentence grammatical? Yes or No: kon teacher brings kar box
A: Yes

Q: Is this sentence grammatical? Yes or No: kars women hold kon newspaper
A:


3it [00:52, 17.62s/it]

Q: Is this sentence grammatical? Yes or No: kon cat eats kon meal
A: Yes

Q: Is this sentence grammatical? Yes or No: kars women bring toy kon
A: No

Q: Is this sentence grammatical? Yes or No: kar girl reads kon chapter
A: Yes

Q: Is this sentence grammatical? Yes or No: kons dogs kon hold toy
A: No

Q: Is this sentence grammatical? Yes or No: takes man kon kar bottle
A: No

Q: Is this sentence grammatical? Yes or No: kars women climb mountain kar
A: No

Q: Is this sentence grammatical? Yes or No: kons take teachers kon cap
A: No

Q: Is this sentence grammatical? Yes or No: girl kar brings kar bottle
A: No

Q: Is this sentence grammatical? Yes or No: kons men bottle kar bring
A: No

Q: Is this sentence grammatical? Yes or No: kar girl brings kar cup
A:


3it [00:56, 18.80s/it]


KeyboardInterrupt: 

In [ ]:
mlp_effects_cache /= len(prompts)
attn_effects_cache /= len(prompts)

mlp_effects_cache = torch.nan_to_num(mlp_effects_cache)
attn_effects_cache = torch.nan_to_num(attn_effects_cache)

flattened_effects_cache = mlp_effects_cache.view(-1)
top_neurons = flattened_effects_cache.topk(k=40)
two_d_indices = torch.cat((((top_neurons[1] // mlp_effects_cache.shape[1]).unsqueeze(1)), ((top_neurons[1] % mlp_effects_cache.shape[1]).unsqueeze(1))), dim=1)

with open(f'/home/gridsan/arunas/broca/llama/llama-attr-patch-scripts/mlp/{sType}.pkl', 'wb') as f:
    pickle.dump(two_d_indices, f)

flattened_effects_cache = attn_effects_cache.view(-1)
top_neurons = flattened_effects_cache.topk(k=40)
two_d_indices = torch.cat((((top_neurons[1] // attn_effects_cache.shape[1]).unsqueeze(1)), ((top_neurons[1] % attn_effects_cache.shape[1]).unsqueeze(1))), dim=1)

with open(f'/home/gridsan/arunas/broca/llama/llama-attr-patch-scripts/attn/{sType}.pkl', 'wb') as f:
    pickle.dump(two_d_indices, f)